In this project, we will fine-tune the hyperparameters of a neural network on fashion mnist dataset. More precisely, we will define a function that builds and compiles an MLP to classify Fashion MNIST images, using hyperparameters such as the number of hidden layers (n_hidden), the number of neurons per layer (n_neurons), the learning rate 
(learning_rate), and the type of optimizer to use (optimizer).

In [3]:
import tensorflow as tf
import keras_tuner as kt

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [4]:
def build_model(hp):
    #define the hyperparameters
    n_hidden = hp.Int('n_hidden', min_value=0, max_value=8, default=2)
    n_neurons = hp.Int('n_neurons', min_value=16, max_value=256)
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    optimizer = hp.Choice('optimizer', values=['sgd', 'adam'])
    if optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

Perform randomized search:

In [5]:
random_search_tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, overwrite=True,
                                     directory='fashion_mnist', project_name='rnd_search', seed=369)
random_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 02m 26s]
val_accuracy: 0.8774000406265259

Best val_accuracy So Far: 0.8774000406265259
Total elapsed time: 00h 09m 16s
INFO:tensorflow:Oracle triggered exit


In [17]:
best_model = random_search_tuner.get_best_models()[0]

best_params = random_search_tuner.get_best_hyperparameters()

In [18]:
best_params[0].values

{'n_hidden': 7,
 'n_neurons': 165,
 'learning_rate': 0.0007592748099523601,
 'optimizer': 'adam'}

In [19]:
random_search_tuner.oracle.get_best_trials()[0].summary()

Trial summary
Hyperparameters:
n_hidden: 7
n_neurons: 165
learning_rate: 0.0007592748099523601
optimizer: adam
Score: 0.8774000406265259


Fit the full training data and evaluate the model on test set:

In [20]:
history = best_model.fit(X_train_full, y_train_full, epochs=10)
test_loss, test_accuracy = model.evaluate(X_test, y_test)

Epoch 1/10
   7/1875 [..............................] - ETA: 17s - loss: 0.2763 - accuracy: 0.8705  

2023-01-08 10:43:24.822586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 14s 7ms/step - loss: 0.3185 - accuracy: 0.8855
Epoch 2/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.3067 - accuracy: 0.8900
Epoch 3/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2971 - accuracy: 0.8920
Epoch 4/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2907 - accuracy: 0.8946
Epoch 5/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2897 - accuracy: 0.8955
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2757 - accuracy: 0.8996
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2808 - accuracy: 0.8983
Epoch 8/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2793 - accuracy: 0.8995
Epoch 9/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2644 - accuracy: 0.9045
Epoch 10/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2598

NameError: name 'model' is not defined

In [ ]:
print(test_loss, test_accuracy)